In [9]:
from bs4 import BeautifulSoup as BS
import requests
import re
import pandas as pd

In [45]:
def superjob_item_parser(item):

    vacancy_date = {}  
    # Название вакансии
    vacancy_name = item.find_all('a')
    if len(vacancy_name) > 1:
        vacancy_name = vacancy_name[-2].getText()
    else:
        vacancy_name = vacancy_name[0].getText()
    vacancy_date['vacancy_name'] = vacancy_name   
    # Название компании
    company_name = item.find('span', {'class': 'f-test-text-vacancy-item-company-name'})    
    vacancy_date['company_name'] = company_name
    # Город
    company_location = item.find('span', {'class': 'f-test-text-company-item-location'}) \
                            .findChildren()[1] \
                            .getText() \
                            .split(',')
    
    vacancy_date['city'] = company_location[0] 
    # Метро
    if len(company_location) > 1:
        metro_station = company_location[1]
    else:
        metro_station = None
    vacancy_date['metro_station'] = metro_station
    # Зарплата
    salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                  .findChildren()
    if not salary:
        salary_min = None
        salary_max = None
        salary_currency = None
    else:
        salary_currency = salary[-1].getText()
        is_check_salary = item.find('span', {'class': 'f-test-text-company-item-salary'}) \
                                .getText() \
                                .replace(u'\xa0', u' ') \
                                .split(' ', 1)[0]
        text = salary[0].getText().replace(u'\xa0', u'')
        if is_check_salary == 'до' and text.isdigit():
            salary_min = None
            salary_max = text
        elif is_check_salary == 'от' and text.isdigit():
            salary_min = text
            salary_max = None
        else:
            salary_min = None
            salary_max = None        
        
    vacancy_date['salary_min'] = salary_min
    vacancy_date['salary_max'] = salary_max
    vacancy_date['salary_currency'] = salary_currency    
    # Связь
    vacancy_link = item.find_all('a')
    
    if len(vacancy_link) > 1:
        vacancy_link = vacancy_link[-2]['href']
    else:
        vacancy_link = vacancy_link[0]['href']
    
    vacancy_date['vacancy_link'] = f'https://www.superjob.ru{vacancy_link }'    
    # Сайт
    vacancy_date['site'] = 'www.superjob.ru'
    
    return vacancy_date

In [46]:
def superjob_parser(vacancy):
    vacancy_date = []
    
    params = {
        'keywords': vacancy, \
        'profession_only': '1', \
        'geo[c][0]': '15', \
        'geo[c][1]': '1', \
        'geo[c][2]': '9', \
        'page': ''
    }
    
    headers = {
        'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64; rv:69.0) Gecko/20100101 Firefox/69.0'
    }
    
    link = 'https://www.superjob.ru/vacancy/search/'
       
    html = requests.get(link, params=params, headers=headers)
    
    if html.ok:
        parsed_html = BS(html.text,'html.parser')
    
        page_block = parsed_html.find('a', {'class': 'f-test-button-1'})
    if not page_block:
        last_page = 1
    else:
        page_block = page_block.findParent()
        last_page = int(page_block.find_all('a')[-2].getText())
    
    for page in range(0, last_page + 1):
        params['page'] = page
        html = requests.get(link, params=params, headers=headers)
        
        if html.ok:
            parsed_html = BS(html.text,'html.parser')
            vacancy_items = parsed_html.find_all('div', {'class': 'f-test-vacancy-item'})
                        
            for item in vacancy_items:
                vacancy_date.append(superjob_item_parser(item))
                
    return vacancy_date

In [47]:
vacancy_date = []
vacancy_date.extend(superjob_parser('Python'))   
df = pd.DataFrame(vacancy_date)

In [48]:
df[0:100]

,vacancy_name,company_name,city,metro_station,salary_min,salary_max,salary_currency,vacancy_link,site
0,Технопарк «Сколково»,[[Технопарк «Сколково»]],•,None,None,None,По договорённости,https://www.superjob.ru/clients/tehnopark-skol...,www.superjob.ru
1,Деловые Ресурсы,[[Деловые Ресурсы]],•,None,None,None,По договорённости,https://www.superjob.ru/clients/delovye-resurs...,www.superjob.ru
2,Python developer,[[Шпак Виктория Сергеевна]],•,None,None,None,месяц,https://www.superjob.ru/vakansii/python-develo...,www.superjob.ru
3,Программист Android / Python (удаленная работа),[[VashHR]],•,None,None,None,месяц,https://www.superjob.ru/vakansii/programmist-a...,www.superjob.ru
4,Сбербанк России,[[Сбербанк России]],•,None,None,None,По договорённости,https://www.superjob.ru/clients/sberbank-rossi...,www.superjob.ru
5,Python Tech Lead,[[Финтех Айкью]],•,None,None,None,месяц,https://www.superjob.ru/vakansii/python-tech-l...,www.superjob.ru
6,Инженер-разработчик (Python),[[НПК РоТеК]],•,None,None,None,По договорённости,https://www.superjob.ru/vakansii/inzhener-razr...,www.superjob.ru
7,Умный Код,[[Умный Код]],•,None,None,None,месяц,https://www.superjob.ru/clients/umnyj-kod-4143...,www.superjob.ru
8,Umbrella IT,[[Umbrella IT]],•,None,None,None,По договорённости,https://www.superjob.ru/clients/umbrella-it-41...,www.superjob.ru
9,Разработчик Python,[[Лиман-трейд]],•,None,None,None,По договорённости,https://www.superjob.ru/vakansii/razrabotchik-...,www.superjob.ru
